1. iterate folder
2. get csv file
3. convert csv to get the x and y of the circles
4. save csv to approriate file name
5. move all newly created csv file to train folder

In [1]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
from simpledbf import Dbf5

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [2]:
'''
process_csv 
update:
20231206 - updated to read dbf instead of csv file


parameter(s):
    in  : 
        1.csv_file - string - path of the dbf file to be processed 
        # csv_file - string - path of the csv file to be processed
        2. image_shape - tuple, width and length of the image - default 1056, 816
    out : dataframe of the processed csv file

csv file sample
======================
FID;FID_Point_;OBJECTID;X_Point;Y_Point;FID_GridIn;PageName;PageNumber;X_Anchor;Y_Anchor;X_Anchor_1;Y_Anchor_1;X_Anchor_2;Y_Anchor_2;X_Anchor_3;Y_Anchor_3
0;19640;19641;736854.093986999940000;9303520.247230000800000;31;A31;1;736592.207902000050000;9303516.227159999300000;736595.050906999970000;9304210.743000000700000;737128.876837000020000;9303514.026709999900000;737131.726303000000000;9304208.544709999100000
1;19643;19644;737045.864260999950000;9303526.252170000200000;31;A31;1;736592.207902000050000;9303516.227159999300000;736595.050906999970000;9304210.743000000700000;737128.876837000020000;9303514.026709999900000;737131.726303000000000;9304208.544709999100000

Anchor_1 is the upper left dot
Anchor_3 is the bottom right dot


'''

def process_csv(csv_file, image_shape=(1056, 816)):
#     csv_raw = pd.read_csv(csv_file, sep=";")
    dbf = Dbf5(csv_file)
    csv_raw = dbf.to_dataframe()
    
    # get the upper left and bottom right coordinates
    x_0 = max(csv_raw['X_Anchor_1'])
    x_1 = max(csv_raw['X_Anchor_2'])
    y_0 = max(csv_raw['Y_Anchor_1'])
    y_1 = max(csv_raw['Y_Anchor_2'])
    
    # get the ratio - width of the image/width of the coordinates
    try:
        scale_x = image_shape[1] / (x_1 - x_0)
        scale_y = image_shape[0] / (y_0 - y_1)
    except ZeroDivisionError as e:
        print("File: {}; Variables: {}, {}".format(csv_file, x_1, x_0))
        
    csv_raw["x"] = csv_raw['X_Point'] - csv_raw['X_Anchor_1']
    csv_raw["y_raw"] = csv_raw["Y_Point"] - csv_raw["Y_Anchor_1"]
    
    csv = csv_raw[csv_raw['y_raw'] < 1][['x','y_raw']]
    csv['y'] = abs(csv['y_raw'])
    
    csv.drop(['y_raw'], axis=1, inplace=True)
    csv.reset_index(inplace=True)
    
    csv['x'] = csv['x'] * scale_x
    csv['y'] = csv['y'] * scale_y
    
    return csv

In [3]:
root = 'D:/Dev/project/roofcounting/'
label = os.path.join(root, "data/csv")        
labelled_paths = []
for path in glob.glob(os.path.join(label, "*.dbf")):
    labelled_paths.append(path)

for label_file in labelled_paths: 
#     print(label_file)
    csv = process_csv(label_file)
    csv_name = label_file.replace("Image_", "/processed/Image_").replace(".dbf", ".csv")
    csv.to_csv(csv_name, index=False)
#     print("Label: {}; csv: {}".format(label_file, csv_name))

In [6]:
im = (2, 3)
print(im[0])

2
